In [49]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from copy import deepcopy

In [2]:
fruits = pd.read_csv('fruit_data_with_colors.csv')

In [3]:
features = fruits.columns[-3:].tolist()
features

['width', 'height', 'color_score']

In [7]:
X = fruits[features].values
Y = fruits['fruit_label'].values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [8]:
[X_train.shape, Y_train.shape]

[(47, 3), (47,)]

In [10]:
[X_test.shape, Y_test.shape]

[(12, 3), (12,)]

In [11]:
[Y_test.min(), Y_test.max()]

[1, 4]

In [29]:
Y_train=Y_train-1

In [30]:
[Y_train.min(), Y_train.max()] # class 0, 1, 2, 3

[0, 3]

In [12]:
Y_test=Y_test-1

In [13]:
[Y_test.min(), Y_test.max()] # class 0, 1, 2, 3

[0, 3]

In [31]:
tree1=DecisionTreeClassifier(max_depth=10)
tree2=DecisionTreeClassifier(max_depth=10)
tree3=DecisionTreeClassifier(max_depth=10)

In [41]:
[X_train.shape, Y_train.shape]

[(47, 3), (47,)]

In [43]:
int(47*0.8)

37

In [50]:
indexlist=np.arange(0, 47)
indexlist

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46])

In [51]:
indexlist=np.arange(0, 47)
np.random.shuffle(indexlist)
indexlist

array([45, 15, 36,  1, 42, 17, 43, 13, 34, 44, 31, 38, 32,  6,  3, 28, 35,
       24, 22,  8, 19,  0, 14, 11, 16, 10, 39,  4, 18, 33, 23,  2, 20, 27,
       25, 40, 41, 37, 12, 29,  5, 21, 30, 46, 26,  9,  7])

In [52]:
indexlist1=deepcopy(indexlist[0:37])

In [53]:
np.random.shuffle(indexlist)
indexlist2=deepcopy(indexlist[0:37])

In [54]:
np.random.shuffle(indexlist)
indexlist3=deepcopy(indexlist[0:37])

In [55]:
tree1.fit(X_train[indexlist1], Y_train[indexlist1])
tree2.fit(X_train[indexlist2], Y_train[indexlist2])
tree3.fit(X_train[indexlist3], Y_train[indexlist3])

DecisionTreeClassifier(max_depth=10)

In [56]:
Y_test_pred1=tree1.predict(X_test)
Y_test_pred2=tree2.predict(X_test)
Y_test_pred3=tree3.predict(X_test)

In [21]:
#wrong: this is for classification
#Y_test_pred=(1/3)*(Y_test_pred1+Y_test_pred2+Y_test_pred3)

In [26]:
#acc_test=(Y_test_pred==Y_test).sum()/len(Y_test)
#acc_test

In [35]:
y_pred_n1

0

In [36]:
counter[y_pred_n1]

3

In [57]:
#majority voting
Y_test_pred=[]
for n in range(X_test.shape[0]):
    #x_n=X_test[n]
    counter=np.array([0, 0, 0, 0])
    y_pred_n1=Y_test_pred1[n]
    y_pred_n2=Y_test_pred2[n]
    y_pred_n3=Y_test_pred3[n]
    counter[y_pred_n1]+=1
    counter[y_pred_n2]+=1
    counter[y_pred_n3]+=1
    y_pred_n=np.argmax(counter)
    Y_test_pred.append(y_pred_n)

In [58]:
Y_test_pred=np.array(Y_test_pred)
Y_test_pred.dtype

dtype('int64')

In [59]:
acc_test=(Y_test_pred==Y_test).sum()/len(Y_test)
acc_test

0.8333333333333334

In [65]:
class MyRandomeForestClassifier:
    def __init__(self, n_estimators, max_depth, max_samples):
        self.n_estimators=n_estimators
        self.max_depth=max_depth
        self.max_samples=max_samples
        #define trees here
        self.tree_list=[]
        for n in range(0, n_estimators):
            self.tree_list.append(DecisionTreeClassifier(max_depth=self.max_depth))
        self.num_classes=None
        
    def fit(self, X, Y):
        #X: (N, M)
        #Y: (N,)
        for n in range(0, self.n_estimators):
            #get a subset from X, Y
            indexlist=np.arange(0, X.shape[0])
            np.random.shuffle(indexlist)
            indexlist=indexlist[0:int(self.max_samples*X.shape[0])]
            self.tree_list[n].fit(X[indexlist], Y[indexlist])
        self.num_classes = Y.max()+1
        
    def predict(self, X):
        #X: (N, M)
        Y_pred_tree_list=[]
        for m in range(len(self.tree_list)):
            Y_pred_tree_m=self.tree_list[m].predict(X)
            Y_pred_tree_list.append(Y_pred_tree_m)
        #majority voting
        Y_pred=[]
        for n in range(X.shape[0]):
            #x_n = X[n]
            counter=np.zeros(self.num_classes)
            for m in range(len(self.tree_list)):
                y_pred_m_n=Y_pred_tree_list[m][n]
                counter[y_pred_m_n]+=1
            y_pred_n=np.argmax(counter)
            Y_pred.append(y_pred_n)
        Y_pred=np.array(Y_pred, dtype='int64')
        return Y_pred

In [66]:
RF=MyRandomeForestClassifier(n_estimators=100, max_depth=10, max_samples=0.8)

In [67]:
RF.fit(X_train, Y_train)

In [68]:
Y_test_pred=RF.predict(X_test)

In [69]:
acc_test=(Y_test_pred==Y_test).sum()/len(Y_test)
acc_test

0.9166666666666666

In [70]:
n_estimators=100

In [ ]:
n_jobs=4 #

In [71]:
100/4

25.0

In [ ]:
#25 run on cpu1
#25 run on cpu2
#25 run on cpu3
#25 run on cpu4